### Imports
All imports occur here

In [1]:
import pandas as pd
import sim.model
from sim import sum_stats as ss
import time
import tskit
import msprime
import allel
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

### Choose parameters
Choose some parameters so it runs relatively quickly:

In [2]:
seq_features = sim.model.SeqFeatures(length=int(20e6), recombination_rate=1.8e-8, mutation_rate=6e-8)

slim_parameters = {
    'pop_size_domestic_1': 1000,  # Population sizes are diploid.
    'pop_size_wild_1': 1000,
    'pop_size_captive': 70,
    'mig_rate_captive': 0.005,  # Migration from wild -> captive
    'mig_length_wild': 20,
    'mig_rate_wild': 0.005,  # Rate of migration from domestic -> wildcats
    'captive_time': 26,  # Time captive population established in SLiM
    }

recapitate_parameters = {
        'pop_size_domestic_2': 2000,
        'pop_size_wild_2': 20000,
        'div_time': 40000,
        'mig_rate_post_split': 0.005,
        'mig_length_post_split': 5000,
        'bottleneck_time_wild': 3000,
        'bottleneck_strength_wild': 20000,
        'bottleneck_time_domestic': 3000,
        'bottleneck_strength_domestic': 20000,
    }

### Run simulation

In [3]:
start_time = time.time()

# Run model
s = sim.model.WildcatSimulation(seq_features=seq_features, random_seed=1)
command = s.slim_command(slim_parameters)
decap_trees = s.run_slim(command)
demographic_events = s.demographic_model(**recapitate_parameters)
tree_seq = s.recapitate(decap_trees, demographic_events)

# Print out useful bits and bobs
print("Simulation finished in {:.2f} s".format(time.time()-start_time))
print("Command ran: {}".format(command))


Simulation finished in 6.39 s
Command ran: slim -d pop_size_domestic_1=1000 -d pop_size_wild_1=1000 -d pop_size_captive=70 -d mig_rate_captive=0.005 -d mig_length_wild=20 -d mig_rate_wild=0.005 -d captive_time=26 -d length=20000000 -d recombination_rate=1.8e-08  -d decap_trees_filename='"../output/decap_1.trees"' -s 40 slim_model.slim


### Sample population

In [4]:
samples = s.sample_nodes(tree_seq, [5, 30, 10])  # Match number of samples to the WGS data
tree_seq = tree_seq.simplify(samples=samples)

In [5]:
data = sim.model.Results(tree_seq)
data.allelify()  # convert from numpy to scikit allel equivilents

In [6]:
ss.elfi_summary(data)

array([[ 5.08400000e+03,  2.20100000e+03,  1.47800000e+03,
         1.34800000e+03,  5.91000000e+02,  3.61416667e+03,
         1.81556667e+04,  1.27966667e+03,  1.18083333e+03,
         1.03116667e+03,  3.35750000e+03,  3.06900000e+03,
         2.97850000e+03,  2.85250000e+03,  2.55750000e+03,
         1.87222222e+02,  2.92666667e+02,  5.30000000e+01,
         2.60222222e+02,  1.48111111e+02,  1.76820074e-04,
         1.95017731e-03,  5.33725157e-04,  1.32862002e-04,
         1.89151500e-04,  1.62517042e-03,  4.17592629e-04,
         8.35169521e-05, -3.27538235e-01,  7.18690245e-01,
         1.16736712e+00,  2.03856038e+00,  2.17978415e-02,
         1.80698807e-01,  7.10902103e-02,  1.38685678e-01,
         2.02536527e-02,  2.44064234e-01,  6.45313216e-02,
         1.67215899e-02,  8.47100000e+03,  1.46442000e+05,
         5.57500000e+03,  1.27514000e+05,  2.68083838e+01,
        -3.60759121e-01,  1.59711760e+00,  5.10021486e-03,
         5.94795488e-03,  1.39082882e-03,  7.91479629e-0

In [6]:
genotypes_array = data.genotypes["domestic"]

In [7]:
genotypes_array_pre_pickle = data.genotypes["domestic"]

In [ ]:
data = sim.model.Results(tree_seq)

In [ ]:
data = sim.model.Results(tree_seq)
data.genotypes = {key: allel.Genotypes(value) for key, value in data.genotypes.items()}  # Convert to allel array
pca_data = ss.pca(data.genotypes["all_pops"].to_n_alt(), data.subpops)

In [ ]:
data.allelify()

In [ ]:
data.genotypes["domestic"]

In [ ]:
data.genotypes["domestic"].__getattr__()

### Calculate summary statistics

In [ ]:
pca_ss = ss.pca_stats(pca_data)
trad_ss = ss.traditional_stats(data)
ld_ss = ss.ld_stats(data)

In [ ]:
import pickle
with open("temp.pickle", "wb") as f:
    pickle.dump(data.genotypes["domestic"], f)
with open("temp.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
ss.collated_summaries([data, data]).shape

In [ ]:
collated_stats = sim.utils.flatten_dict(collated_stats)

### Plot PCA

In [ ]:
sns.scatterplot(pca_data["pc1"], pca_data["pc2"], hue=pca_data["population"])
plt.legend(bbox_to_anchor=(1.05, 1))

### Check tsinfer ancestral state irrelevant toy example

In [ ]:
import joblib
joblib.dump(np.array(genotypes), "../output/test_genotypes.joblib")
joblib.dump(pos, "../output/test_pos.joblib")

In [ ]:
df = pd.DataFrame({"AB": [1,2,3], "AC": [5,4,5], "CC": [5,7,5]})

In [ ]:
[col for col in list(df) if "A" in col]

In [ ]:
positions = np.loadtxt("../data/e3.012.pos", delimiter="\t", usecols=1)
genotypes = np.loadtxt("../data/e3.012", delimiter="\t", usecols=range(1, len(positions)+1))
genotypes = genotypes.T
assert len(positions) == genotypes.shape[0]

# For now just assume that missings are ancestral
genotypes[genotypes == -1] = 0


In [ ]:
# Cam read in with scikit allel but genotypes looks dodge
callset = allel.read_vcf("../data/e3.vcf")
pos = callset["variants/POS"]
genotypes = allel.GenotypeArray(callset["calldata/GT"])

In [ ]:
callset["samples"][0:3]

In [ ]:
sample_info = pd.read_csv("../data/e3_sample_info.csv", usecols=["NAME", "SOURCE"])

# Ensure that individuals are in same order (after 012 conversion)
assert np.all(sample_info["NAME"] == np.loadtxt("../data/e3.012.indv", dtype=str))

pca_pipeline(genotypes, pos, sample_info["SOURCE"].to_list())

In [ ]:
genotypes, pos = ss.maf_filter(genotypes, pos)
genotypes, pos = ss.ld_prune(genotypes.to_n_alt(), pos)

In [ ]:
plot_ld(genotypes[:1000], 'Pairwise LD after pruning')

### Plot both

In [ ]:
sample_population = np.asarray(["domestic"]*4 + ["wild"]*45 + ["captive"]*46)
populations = ["domestic", "wild", "captive"]
pop_colours = ["#FF0000", "#FFA500", "#0000FF"]

# Simulated data pca
coords, model = allel.pca(genotypes, n_components=10, scaler='patterson')
sim_df = pd.DataFrame({"pc1": coords[:, 0], "pc2": coords[:, 1],
                       "population": sample_population, "simulated_or_observed": "simulated"})

# Real data pca
real_genotypes = np.loadtxt("../data/snps.012", delimiter=" ", skiprows=1)
real_genotypes = real_genotypes[:,1:].transpose()  # Get rid of index and convert individuals to columns
coords, model = allel.pca(real_genotypes, n_components=2, scaler='patterson')
real_df = pd.DataFrame({"pc1": coords[:, 0], "pc2": coords[:, 1],
                   "population": sample_population, "simulated_or_observed": "observed"})

# Combined data
combined_df = sim_df.append(real_df)

In [ ]:
sns.set(style='darkgrid', font_scale=1.3)

g = sns.relplot(x="pc1", y="pc2",
                row="simulated_or_observed", hue="population",
                kind="scatter", data=combined_df,
                facet_kws=dict(sharex=False, sharey=False),
                aspect=1.4)

axes = g.axes.flatten()
axes[0].set_title("Simulated")
axes[1].set_title("Observed")

#g.savefig("../plots/simulated_vs_observed_pca.png", dpi=600)

### Scaling summary statistics

In [ ]:
stats = pd.read_csv("../output/summary_stats.csv")
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_sum_stats = scaler.fit_transform(stats)
scaler.inverse_transform(scaled_sum_stats)